<a href="https://colab.research.google.com/github/jlee932/congressional-analysis/blob/main/GSB521_Group_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# hypothesis: In combination with the party, we beleive the 25-44 age group earns more income than the 25 and under and 44-64 bracket.
#add new data to test this hypothesis 
#notbook has to have to the analysis (in support or not supposting our hypothesis)

#Links for definition of "Householder"
#https://www.census.gov/programs-surveys/cps/technical-documentation/subject-definitions.html#:~:text=is%20not%20related.-,Householder,%2C%20boarders%2C%20or%20paid%20employees.
#https://www2.census.gov/programs-surveys/acs/tech_docs/user_notes/Family_Equalization_2006.pdf


# We begin by importing our libraries and the ACS data. 

In [ ]:
# first some initial setup of libraries we will be using
import pandas as pd # this is the main library used in python for manipulating data. 
import json # this is a library to parse json (javascript object notation)
from urllib.request import urlopen # library to read data from a URL
import matplotlib.pyplot as plt # visualization library we will use that integrates with pandas


In [ ]:
# We are reading in the data we want from ACS 2021. The API returns json data. Try copy and pasting the link yourself in a browser to see what the raw data looks like.
url="https://api.census.gov/data/2021/acs/acs1?get=NAME,B19001_001E,B19001_002E,B19001_017E&for=congressional%20district:*&in=state:*"

# store the response of URL
response = urlopen(url)

# storing the JSON response from url in data
data_json = json.loads(response.read())
data_json.pop(0) # the first row of data is header information which we don't need, so pop removes that from the dataset.

# Hand populate the columns based on the variable descriptions census.gov documents here: https://api.census.gov/data/2021/acs/acs1/variables.html. Note they correspond back to the variable names I included in the URL above.
cols = ['District Name','Total Households','Less than $10k', '$200k+', 'State code', 'district number']

acs_df = pd.DataFrame(data_json, columns = cols)

# time to do some basic cleanup of the data. We will need to extract the district number and state in a regular format so we can later join with the party data.
acs_df[['District', 'State']] = acs_df['District Name'].str.split(',', expand=True)
acs_df['district number'] = pd.to_numeric(acs_df['district number']) #Turn state into a numeric 

acs_df['State'] = acs_df['State'].str.strip() # Strips the white space 

# Let's just keep the columns we need
acs_df = acs_df[['Total Households', 'Less than $10k', "$200k+", 'district number', 'State']]

# take a look at the top results in the dataset
acs_df.head() #Looks at the top 5 of the data 

In [ ]:
# Convert the number columns into number data types so we can math on them
acs_df["Total Households"] = pd.to_numeric(acs_df["Total Households"])
acs_df["Less than $10k"] = pd.to_numeric(acs_df["Less than $10k"])
acs_df['$200k+'] = pd.to_numeric(acs_df['$200k+'])

# Compute some new columns based on the existing columns for later visualization
acs_df['% of households over $200k'] = (100*acs_df['$200k+'])/acs_df['Total Households']
acs_df['% of households less than $10k'] = (100*acs_df['Less than $10k'])/acs_df['Total Households'] 

# now let's look at the data again
acs_df.head()

#Importing Congressional data

In [ ]:
# read from the congressional data and put into a pandas dataframe
party_df = pd.read_csv("http://goodcsv.com/wp-content/uploads/2020/08/us-house-of-representatives-2020.csv", encoding = "ISO-8859-1")

# extract the district number from the data (it was in the format of 5th and we want that to just be 5) using a regular expression.
party_df['district number'] = party_df['District/Position'].str.extract('(\d+)')
party_df['district number'] = party_df['district number'].fillna(0)
party_df['district number'] = pd.to_numeric(party_df['district number'])
party_df['State'] = party_df['State/Territory']
party_df['State'] = party_df['State'].str.strip()
party_df['Party'] = party_df['Party'].str.strip() # remove extraneous whitespace

# Let's just keep the columns we need
party_df = party_df[['State', 'Party', "district number"]]

party_df.head(5)

In [ ]:
# Quick check to see if the data is correct 
party_df['State'].value_counts()

In [ ]:
# Merge is like a sql natural join. We are joining on the State and District numbers being the same between the two sets. Similar to natural join.
merged_df = pd.merge(acs_df, party_df, on=["State", "district number"])

#Importing Householder data 

In [ ]:
# We are reading in the data we want from ACS 2021. The API returns json data. Try copy and pasting the link yourself in a browser to see what the raw data looks like.
url2 = "https://api.census.gov/data/2021/acs/acs1?get=NAME,B19037_001E,B19037_002E,B19037_019E,B19037_036E&for=congressional%20district:*&in=state:*"

response2 = urlopen(url2)

data_json2 = json.loads(response2.read())
data_json2.pop(0)


In [ ]:
cols2 =  ['District Name','Total Households','Householder under 25','Householder 25 to 44','Householder 44 to 65','State code','district number'] #where do we find the names of the columns? Named off the description in the asi website 

acs_df2 = pd.DataFrame(data_json2,columns=cols2)

acs_df2[['District', 'State']] = acs_df2['District Name'].str.split(',', expand=True)
acs_df2['district number'] = pd.to_numeric(acs_df2['district number'])

acs_df2['State'] = acs_df2['State'].str.strip() # Strips the white space

#acs_df2['Total Households'] = acs_df2['Householder under 25'] + acs_df2['Householder 25 to 44'] + acs_df2['Householder 44 to 65'] + acs_df2['State code'] + acs_df2['district number']

# Keeping the columns we need
acs_df2 = acs_df2[['Total Households','Householder under 25','Householder 25 to 44','Householder 44 to 65','State','district number']]

# Looking at the top 5 of our data
acs_df2.head()  


In [ ]:
acs_df2['State'].value_counts()

#Merging all 3 data sets

In [ ]:
# from pandas.core.reshape.merge import merge
# merged_df = pd.merge(acs_df, party_df, on=["State", "district number"])

merge3 = merge(merged_df,acs_df2)


ValueError: ignored

#Checking if data sets were merged correctly

In [ ]:
merge3.describe()

There are 435 total rows, which matches up with the number of congress members in the 116th congress. 

In [ ]:
merge3['Party'].value_counts()

# Exploritory analysis to test our hypothesis.

The following scatterplot is of households earning over 200k and
10k on the other axis. We will color the plot based upon the party affiliation of the districts congress member as of the 116th congress.

In [ ]:
R = merged_df[merged_df['Party'].str.contains('R')]
D = merged_df[merged_df['Party'].str.contains('D')]
L = merged_df[merged_df['Party'].str.contains('L')]

fig, ax = plt.subplots(figsize=(15, 12))
R.plot.scatter(x='% of households over $200k', y='% of households less than $10k', c='red',label='Republican', ax=ax)
D.plot.scatter(x='% of households over $200k', y='% of households less than $10k',c='blue',label='Democrat', ax=ax)
L.plot.scatter(x='% of households over $200k', y='% of households less than $10k',c='gold',label='Libertarian', ax=ax)

#for i, txt in enumerate(df['State']):
#   ax.annotate(txt, (df['% of households over $200k'].iat[i], df['% of households less than $10k'].iat[i]))

plt.legend()
plt.show()

We'll begin by looking at the percentage of households that earn over $200k and their householder is under the age of 25.

In [ ]:
R = merge3[merge3['Party'].str.contains('R')]
D = merge3[merge3['Party'].str.contains('D')]
L = merge3[merge3['Party'].str.contains('L')]

fig, ax = plt.subplots(figsize=(15, 12))
R.plot.scatter(x='% of households over $200k', y='Householder under 25', c='red',label='Republican', ax=ax)
D.plot.scatter(x='% of households over $200k', y='Householder under 25',c='blue',label='Democrat', ax=ax)
L.plot.scatter(x='% of households over $200k', y='Householder under 25',c='gold',label='Libertarian', ax=ax)


Percentage of households earning over $200k and their householder  is between the ages of 25 to 44. 

In [ ]:
R = merge3[merge3['Party'].str.contains('R')]
D = merge3[merge3['Party'].str.contains('D')]
L = merge3[merge3['Party'].str.contains('L')]


fig, ax = plt.subplots(figsize=(15, 12))
R.plot.scatter(x='% of households over $200k', y='Householder 25 to 44', c='red',label='Republican', ax=ax)
D.plot.scatter(x='% of households over $200k', y='Householder 25 to 44',c='blue',label='Democrat', ax=ax)
L.plot.scatter(x='% of households over $200k', y='Householder 25 to 44',c='gold',label='Libertarian', ax=ax)




Percentage of households earning over $200k and their householder is between the ages of 44 to 65. 

In [ ]:
R = merge3[merge3['Party'].str.contains('R')]
D = merge3[merge3['Party'].str.contains('D')]
L = merge3[merge3['Party'].str.contains('L')]


fig, ax = plt.subplots(figsize=(15, 12))
R.plot.scatter(x='% of households over $200k', y='Householder 44 to 65', c='red',label='Republican', ax=ax)
D.plot.scatter(x='% of households over $200k', y='Householder 44 to 65',c='blue',label='Democrat', ax=ax)
L.plot.scatter(x='% of households over $200k', y='Householder 44 to 65',c='gold',label='Libertarian', ax=ax)



Next, we look percentages of households that earn less than 10k and their householder is under the age of 25.

In [ ]:
R = merge3[merge3['Party'].str.contains('R')]
D = merge3[merge3['Party'].str.contains('D')]
L = merge3[merge3['Party'].str.contains('L')]

fig, ax = plt.subplots(figsize=(15, 12))
R.plot.scatter(x='% of households less than $10k', y='Householder under 25', c='red',label='Republican', ax=ax)
D.plot.scatter(x='% of households less than $10k', y='Householder under 25',c='blue',label='Democrat', ax=ax)
L.plot.scatter(x='% of households less than $10k', y='Householder under 25',c='gold',label='Libertarian', ax=ax)

#R.plot.scatter(x='Householder 44 to 65', y='% of households less than $10k', c='red',label='Republican', ax=ax)
#D.plot.scatter(x='Householder 44 to 65', y='% of households less than $10k',c='blue',label='Democrat', ax=ax)
#L.plot.scatter(x='Householder 44 to 65', y='% of households less than $10k',c='gold',label='Libertarian', ax=ax)

Percentage of households earning less than 10k and their householder is between the ages of 25 to 44.

In [ ]:
R = merge3[merge3['Party'].str.contains('R')]
D = merge3[merge3['Party'].str.contains('D')]
L = merge3[merge3['Party'].str.contains('L')]

fig, ax = plt.subplots(figsize=(15, 12))
R.plot.scatter(x='% of households less than $10k', y='Householder 25 to 44', c='red',label='Republican', ax=ax)
D.plot.scatter(x='% of households less than $10k', y='Householder 25 to 44',c='blue',label='Democrat', ax=ax)
L.plot.scatter(x='% of households less than $10k', y='Householder 25 to 44',c='gold',label='Libertarian', ax=ax)

Lastly, we looked at households earning less than 10k and their householder is between the ages of 44 to 65. 

In [ ]:
R = merge3[merge3['Party'].str.contains('R')]
D = merge3[merge3['Party'].str.contains('D')]
L = merge3[merge3['Party'].str.contains('L')]


fig, ax = plt.subplots(figsize=(15, 12))
R.plot.scatter(x='% of households less than $10k', y='Householder 44 to 65', c='red',label='Republican', ax=ax)
D.plot.scatter(x='% of households less than $10k', y='Householder 44 to 65',c='blue',label='Democrat', ax=ax)
L.plot.scatter(x='% of households less than $10k', y='Householder 44 to 65',c='gold',label='Libertarian', ax=ax)

Conclusion: Based on our analysis we found that their is no real correlation between the age of a householder, and their household income. 